In [1]:
import numpy as np
import torch 
from torchvision import datasets, transforms
from utils import * # get the dataset
from pyhessian import hessian # Hessian computation
from density_plot import get_esd_plot # ESD plot
from pytorchcv.model_provider import get_model as ptcv_get_model # model

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pdb
%matplotlib inline

In [2]:
# enable cuda devices
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Define the input and target
x = torch.linspace(-2, 2, 100).view(-1,1)
target = 2 * x ** 3 - 4 * x ** 2 + 3 * x + torch.randn_like(x)

# Define the model
model = torch.nn.Linear(1, 1)
# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0)
criterion = torch.nn.MSELoss()


for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    output = model(x)
    target = target.view(-1,1)
    loss = criterion(output, target)

    # Backward pass
    loss.backward()
    # crr_bias_list.append(bias.item())
    optimizer.step()

In [4]:
# change the model to eval mode to disable running stats upate
model.eval()

# get dataset 
# train_loader, test_loader = getData()

# for illustrate, we only use one batch to do the tutorial
for inputs, targets in train_loader:
    break

# we use cuda to make the computation fast
model = model.cuda()
inputs, targets = inputs.cuda(), targets.cuda()

NameError: name 'train_loader' is not defined

In [ ]:
# create the hessian computation module
pdb.set_trace()
hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=True)

In [ ]:
# get the top1, top2 eigenvectors
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()

In [ ]:
# This is a simple function, that will allow us to perturb the model paramters and get the result
def get_params(model_orig,  model_perb, direction, alpha):
    for m_orig, m_perb, d in zip(model_orig.parameters(), model_perb.parameters(), direction):
        m_perb.data = m_orig.data + alpha * d
    return model_perb

In [ ]:
# lambda is a small scalar that we use to perturb the model parameters along the eigenvectors 
lams1 = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb1 = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb1.eval()
model_perb1 = model_perb1.cuda()

for lam1 in lams1:
    model_perb1 = get_params(model, model_perb1, top_eigenvector[0], lam1)
    loss_list.append(criterion(model_perb1(inputs), targets).item())   

loss_list = np.array(loss_list)
                         
plt.plot(lams1, loss_list)
plt.ylabel('Loss')
plt.xlabel('w')
plt.title('Loss landscape perturbed based on top Hessian eigenvector')

#landscape.view_init(elev=15, azim=75)
plt.show()